# Data Manipulation for Insight

![](https://www.causact.com/graphics/prodCat.jpg)


> I just got word that the CEO of ZappTech is thinking about hiring our consulting firm. Apparently, his category managers are refusing to talk to one another; acting as if the four product categories are isolated kingdoms.

> He is convinced that ZappTech’s customers shop across multiple categories and thinks they expect the same level of customer service regardless of the product categories represented on their order. Since he doesn’t trust his own team to put effort towards integrating management of the categories, the CEO has provided us data and asked us to investigate two questions: 1) Does service level (measured by on-time shipments) vary across product categories? and 2) how often do orders include products from more than one product category.

## Challenge accepted, let's get the data - DATA LOADING

Notice how `parse_dates` is used to ensure data gets imported as a date object.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import numpy as np

## get primarys cast in the primaries as of February 25th, 2016.
shipDF = pd.read_csv("https://raw.githubusercontent.com/flyaflya/persuasive/main/shipments.csv", parse_dates=['plannedShipDate','actualShipDate'])
prodLineDF = pd.read_csv("https://raw.githubusercontent.com/flyaflya/persuasive/main/productLine.csv")

In [ ]:
## reduce size or it crashed FREE Google Colab
shipDF = shipDF.head(4000)

To answer the CEO’s questions, we will approach the data analysis in four phases:
1.   Data Loading (done above): Make the data available in an data frame with all columns associated with the correct column class.
2.   Lateness Calculation: Define how to measure lateness.
3.   Bring in product category information: In this phase, we will learn to merge the delivery information with the product category information.
4.   Answer the CEO’s questions: Does service level vary by product category? Do we ship items from multiple product categories?

## Data Loading into xarray

In [ ]:
shipLineItemDS = (
    shipDF
    # # chain this dataframe to an xarray function using the pipe() method
    .pipe(xr.Dataset.from_dataframe) ## pipe is panda function that passes df
    #                                  ## as first argument to the xarray function
    #                                  ## which creates a dataset from a df
)

shipLineItemDS

<xarray.Dataset>
Dimensions:          (index: 4000)
Coordinates:
  * index            (index) int64 0 1 2 3 4 5 ... 3994 3995 3996 3997 3998 3999
Data variables:
    shipID           (index) int64 10001 10002 10003 10004 ... 10762 10763 10763
    plannedShipDate  (index) datetime64[ns] 2013-11-06 2013-10-15 ... 2013-10-15
    actualShipDate   (index) datetime64[ns] 2013-10-04 2013-10-04 ... 2013-10-15
    partID           (index) object 'part92b16c5' 'part66983b' ... 'part3e5bca'
    quantity         (index) int64 6 2 1 1 6 15 2 1 1 1 ... 1 3 3 2 1 1 4 1 7 7

Let's create a dataset with just the high-level shipment information.  We will want one row per shipID with plannedShipDate and actualShipDate.

In [ ]:
shipDS = (
    shipLineItemDS
    .get(["shipID","plannedShipDate","actualShipDate"])
    .groupby("shipID")
    .first() ## aggregation function to be used in combo with groupby
)

shipDS

<xarray.Dataset>
Dimensions:          (shipID: 763)
Coordinates:
  * shipID           (shipID) int64 10001 10002 10003 ... 10761 10762 10763
Data variables:
    plannedShipDate  (shipID) datetime64[ns] 2013-11-06 ... 2013-10-15
    actualShipDate   (shipID) datetime64[ns] 2013-10-04 ... 2013-10-15

In [ ]:
prodLineDF

,partID,productLine,prodCategory
0,part0a7f7c6,line7a,Machines
1,part84778b6,line7a,Machines
2,part330b1c9,line6d,Machines
3,parta4ebc9b,line6d,Machines
4,partcf299b0,line6d,Machines
...,...,...,...
11996,part89b1c2,line55,SpareParts
11997,partba7e11e,line55,SpareParts
11998,partfed2e6d,line55,SpareParts
11999,part948c35b,line55,SpareParts


In [ ]:
prodLineDS = (
    prodLineDF
    .set_index("partID") ## this index will be dimension of new dataset
    ## think of the dimension as answering what data do we want to look things up by?
    # # chain this newly indexed dataframe to a function using the pipe() method
    .pipe(xr.Dataset.from_dataframe) ## pipe is panda function that passes df
    #                                  ## as first argument to the xarray function
    #                                  ## which creates a dataset from a df
)

prodLineDS

prodLineDS.to_dataframe().sample(5)

,productLine,prodCategory
partID,,
partb5e6a7d,linec1,Marketables
part01003d2,linea3,Marketables
partcf299b0,line6d,Machines
part08e853,linec1,Marketables
part11d48ae,linea3,Marketables


## Lateness Calculation

Notice that the CEO’s question talks about on-time shipments. We need to be mathematically precise in translating the CEO’s real-world concerns to mathematical calculations; did he really mean shipments, or perhaps orders, or maybe even partID’s?

> **As an analyst, it is your job to form an opinion and validate that opinion with your stakeholder about how you plan to translate real-world concerns into mathematical constructs. Do not immediately fire off an email everytime you have a question; spend some time thinking and researching the issue before you make yourself look silly by asking simplistic questions that waste time. Also, when thinking about an issue, adopting the customer’s perspective is often a good starting point.**

After deliberating, forming an opinion, and validating that opinion, here is what we will assume about measuring lateness at ZappTech:

* Ideally look at customer orders (i.e. orderID), but since we do not have that data and it is rare that an order gets broken into mulitple shipments, using shipID as the observational unit should give a good estimate/proxy of on-time order performance.
* Measuring lateness using quantity does not make sense for ZappTech. Some products, like latex gloves, get ordered by the hundreds whereas machines get ordered one or two at a time.
* Measuring lateness by partID might make sense for evaluating inventory policies on specific parts, but for now talking about lateness by shipID is preferable.
* For each unique shipID, if actualShipDate > plannedShipDate, then the shipID is considered late.

In [ ]:
(
    shipDS
)

<xarray.Dataset>
Dimensions:          (shipID: 763)
Coordinates:
  * shipID           (shipID) int64 10001 10002 10003 ... 10761 10762 10763
Data variables:
    plannedShipDate  (shipID) datetime64[ns] 2013-11-06 ... 2013-10-15
    actualShipDate   (shipID) datetime64[ns] 2013-10-04 ... 2013-10-15

In [ ]:
(
    shipDS
    .assign(lateFlag = shipDS.actualShipDate > shipDS.plannedShipDate)
)

<xarray.Dataset>
Dimensions:          (shipID: 763)
Coordinates:
  * shipID           (shipID) int64 10001 10002 10003 ... 10761 10762 10763
Data variables:
    plannedShipDate  (shipID) datetime64[ns] 2013-11-06 ... 2013-10-15
    actualShipDate   (shipID) datetime64[ns] 2013-10-04 ... 2013-10-15
    lateFlag         (shipID) bool False False False False ... False False False

And now, we take advantage of the fact that Python treats logical (True/False) values as numbers when used with numeric functions. `True` is converted to 1 and `False` converted to 0. Thus, as a simple example of this, we have:

In [ ]:
logicalVector = [True, True, False, False, False]

sum(logicalVector)

2

Notice that the mean of a logical vector is the proportion of `True` values:

In [ ]:
np.mean(logicalVector)

0.4

For calculating late shipments, the following code collapses the data on 23,339 shipments into one row, the mean of our indicator function which represent the average percent late.

In [ ]:
(
    shipDS
    .assign(lateFlag = shipDS.actualShipDate > shipDS.plannedShipDate)
    .mean()
)

<xarray.Dataset>
Dimensions:   ()
Data variables:
    lateFlag  float64 0.0498

We now have a lateness calculation complete, 8.31% of shipments are being delivered later than planned. To retrieve the actual value, we could have added `.lateFlag.item()` to the above method chain.

## Bringing in product category information
The information contained in shipDF does not include product category information. This information happens to be in another table.

So now, we want to calculate lateness by product category, but the product category information is in prodLineDS and the actual/planned shipment data is in shipLineItemDS. How might we combine the information from these two tables?

## TO BE CONTINUED IN PART 2